In [97]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import csv

Code to pull in data from Wiki page

In [98]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

Code to pull in table from Wiki page

In [99]:
table = soup.find('tbody')

Code to create a list with the table headings

In [100]:
row = ['Postcode','Borough','Neighbourhood']

Code to add the data from the table to the list and converts the html data to text

In [101]:
for trs in table.find_all('tr'):
    for td in trs.find_all('td'):
        td_text = td.text
        row.append(td_text)

Code to format the list into a list with groups of three 

In [102]:
n=3
final = [row[i * n:(i + 1) * n] for i in range((len(row) + n - 1) // n )]  

Code to convert the list to a dataframe

In [103]:
df_final = pd.DataFrame(final)
df_final[2] = df_final[2].apply(lambda x:x.strip('\n'))

Code to remove the "Not assigned" rows in the Borough column

In [104]:
df_final = df_final[df_final[1]!='Not assigned']

Code to replace "Not assigned" in Neighborhood column with value in Borough column

In [105]:
df_final[2] = np.where(df_final[2]=='Not assigned', df_final[1], df_final[2])

Code to move the header names from row 0 and makes them the header  

In [106]:
header = df_final.iloc[0]
df_final = df_final[1:]
df_final = df_final.rename(columns = header)

Code that combines Neighborhood names with the same PostalCode and Borough

In [107]:
df_final = df_final.groupby(['Postcode','Borough']).agg({'Neighbourhood': lambda x:','.join(set(x))})

Code that resets the indexes

In [108]:
df_final = df_final.reset_index()

Prints the header and values of the table and prints the shape of table

In [122]:
#print(df_final)
#df_final.shape

In [110]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adds latitude and longitude and filters out those Boroughs with Toronto in them

In [112]:
df_final2 = pd.merge(df_final,df_data,left_on='Postcode',right_on='Postcode')
df_final2 = df_final2.rename(columns={'Latitude_y':'Latitude'})
df_final2 = df_final2[df_final2['Borough'].str.contains('Toronto')]
df_final2 = df_final2.reset_index()

df_final2.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [43]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported


In [44]:
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print ('Libraries imported')

Libraries imported


For Square Data

In [25]:
#toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [45]:
# The code was removed by Watson Studio for sharing.

Getting the latitude and longitudes for Toronto neighbourhoods

In [46]:
toronto_latitude = df_final2['Latitude']
toronto_longitude = df_final2['Longitude']
toronto_neighbourhood = df_final2['Neighbourhood']

#print(toronto_latitude)
#print(toronto_longitude)
#print(toronto_neighbourhood)

Foursquare venues - will get 4 venues per neighborhood from Foursquare and create a dataframe with neighborhood, neighborhood lat, neighborhood long, venue, venue lat, venue long, and venue category.

In [69]:
LIMIT = 10
radius = 500
results = list()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

nearby_venues_final = pd.DataFrame(columns = ['name','categories','lat','lng'])     
    
foursquare_data = list()
for lat, long, neig in zip(toronto_latitude, toronto_longitude, toronto_neighbourhood):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT)
    
    foursquare = requests.get(url).json()
    
    venues = foursquare['response']['groups'][0]['items']

    nearby_venues = json_normalize(venues)
    filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]

    nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)

    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues['Neighbourhood'] = neig
    nearby_venues['N_Latitude'] = lat
    nearby_venues['N_Longitude'] = long
    nearby_venues_final = nearby_venues_final.append(nearby_venues)
    
nearby_venues_final.rename(columns={'categories':'Categories','lat':'V_Latitude','lng':'V_Longitude','name':'Venue'},inplace=True)
nearby_venues_final.reset_index()
nearby_venues_final.head()


,N_Latitude,N_Longitude,Neighbourhood,Categories,V_Latitude,V_Longitude,Venue
0,43.676357,-79.293031,The Beaches,Trail,43.676821,-79.293942,Glen Manor Ravine
1,43.676357,-79.293031,The Beaches,Health Food Store,43.678879,-79.297734,The Big Carrot Natural Food Market
2,43.676357,-79.293031,The Beaches,Pub,43.679181,-79.297215,Grover Pub and Grub
3,43.676357,-79.293031,The Beaches,Neighborhood,43.680563,-79.292869,Upper Beaches
0,43.679557,-79.352188,"Riverdale,The Danforth West",Greek Restaurant,43.677621,-79.351434,Pantheon


Count the number of unique categories.

In [72]:
print('There are {} uniques categories.'.format(len(nearby_venues_final['Categories'].unique())))

There are 116 uniques categories.


Use one hot encoding to create a dataframe that shows the categories by neighborhood and takes the mean of the number of each category in each neighborhood.

In [74]:
toronto_onehot = pd.get_dummies(nearby_venues_final[['Categories']],prefix="",prefix_sep="")
toronto_onehot['Neighbourhood'] = nearby_venues_final['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,Organic Grocery,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Print each neighborhood with the top five most common venues

In [77]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print('----'+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
               venue  freq
0       Concert Hall   0.1
1  French Restaurant   0.1
2       Liquor Store   0.1
3         Steakhouse   0.1
4           Beer Bar   0.1


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop   0.2
1                     Bar   0.1
2  Furniture / Home Store   0.1
3      Italian Restaurant   0.1
4               Pet Store   0.1


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0        Brewery   0.1
1  Auto Workshop   0.1
2     Skate Park   0.1
3  Burrito Place   0.1
4     Restaurant   0.1


----Cabbagetown,St. James Town----
                 venue  freq
0                 Café   0.2
1    Indian Restaurant   0.1
2        Jewelry Store   0.1
3              Butcher   0.1
4  Japanese Restaurant   0.1


----Central Bay Street----
                        venue  freq
0                 Coffee Shop   0.3
1  Modern European Restaurant   0.1
2          Italian Restaura

Function to sort the venues in decending order anda data frame to display the top 10 venues for each neighborhood.

In [143]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Park,Steakhouse,Farmers Market,Liquor Store,Concert Hall,Museum,Cocktail Bar,Beer Bar,French Restaurant,Vegetarian / Vegan Restaurant
1,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Pet Store,Bar,Gym,Italian Restaurant,Caribbean Restaurant,Café,Breakfast Spot,Furniture / Home Store,Auto Workshop
2,Business Reply Mail Processing Centre 969 Eastern,Garden Center,Brewery,Restaurant,Farmers Market,Fast Food Restaurant,Burrito Place,Auto Workshop,Pizza Place,Comic Shop,Skate Park
3,"Cabbagetown,St. James Town",Café,Italian Restaurant,Restaurant,Jewelry Store,Indian Restaurant,Diner,Bakery,Japanese Restaurant,Butcher,Creperie
4,Central Bay Street,Coffee Shop,Gastropub,Sushi Restaurant,Italian Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Ramen Restaurant,Park,Asian Restaurant,College Gym


Clustering the Neighborhoods

In [145]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state = 0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([2, 2, 2, 1, 2, 1, 2, 1, 2, 2], dtype=int32)

Create a dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [146]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
toronto_merged = df_final2

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on='Neighbourhood')
toronto_merged.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Trail,Pub,Health Food Store,Dessert Shop,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall
1,41,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,2,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Pub,Brewery,Italian Restaurant,Food,Flea Market,College Gym
2,42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Park,Gym,Ice Cream Shop,Burger Joint,Pub,Sushi Restaurant,Brewery,Italian Restaurant,Fish & Chips Shop,Liquor Store
3,43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Café,Neighborhood,Bookstore,Ice Cream Shop,Sandwich Place,Fish Market,Cheese Shop,Gay Bar,Concert Hall
4,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Diner,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Concert Hall,Convenience Store


Create Map of the clustering

In [158]:
#map_clusters = folium.Map(location = [toronto_merged['Latitude'],toronto_merged['Longitude']],zoom_start=11)

map_clusters = folium.Map(location = [43.653226,-79.3831843],zoom_start=12)


x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+' Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters
